In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Add scripts directory to path
sys.path.insert(0, '../scripts')

from query_temperature import (
    load_deployment_data,
    load_sensor_data,
    load_time_range_data,
    list_deployments,
    list_sensors
)

## Configuration

In [ ]:
# Path to your database
DB_PATH = '/Users/abdullaalhosani/Projects/highz/temperature.db'

# Deployment to analyze
DEPLOYMENT = 'Adak_2025Dec'

## 1. List Available Data

In [ ]:
# List all deployments
deployments = list_deployments(DB_PATH)
print("Available deployments:")
display(deployments)

In [ ]:
# List all sensors
sensors = list_sensors(DB_PATH)
print("Available sensors:")
display(sensors)

## 2. Load Deployment Data

In [ ]:
# Load all temperature data for a deployment
df = load_deployment_data(DB_PATH, DEPLOYMENT)

print(f"Loaded {len(df):,} temperature readings")
print(f"Unique sensors: {df['sensor_registration'].nunique()}")
print(f"Time range: {df['time_utc_iso'].min()} to {df['time_utc_iso'].max()}")
print(f"Temperature range: {df['value_c'].min():.1f}°C to {df['value_c'].max():.1f}°C")

df.head()

## 3. Basic Statistics

In [ ]:
# Temperature statistics by sensor
stats = df.groupby('sensor_registration')['value_c'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max')
]).round(2)

print("Temperature statistics by sensor:")
display(stats)

## 4. Plot Temperature Over Time

In [ ]:
# Convert UTC to datetime for plotting
df['datetime_utc'] = pd.to_datetime(df['time_utc'], unit='s', utc=True)

# Plot each sensor
plt.figure(figsize=(14, 6))

for sensor in df['sensor_registration'].unique():
    sensor_data = df[df['sensor_registration'] == sensor]
    label = sensor_data['sensor_label'].iloc[0] if sensor_data['sensor_label'].iloc[0] else sensor
    plt.plot(sensor_data['datetime_utc'], sensor_data['value_c'], label=label, alpha=0.7)

plt.xlabel('Time (UTC)')
plt.ylabel('Temperature (°C)')
plt.title(f'Temperature Data - {DEPLOYMENT}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Compare Sensors

In [ ]:
# Temperature distribution by sensor
plt.figure(figsize=(12, 6))

for i, sensor in enumerate(df['sensor_registration'].unique(), 1):
    sensor_data = df[df['sensor_registration'] == sensor]
    label = sensor_data['sensor_label'].iloc[0] if sensor_data['sensor_label'].iloc[0] else sensor
    plt.subplot(1, len(df['sensor_registration'].unique()), i)
    plt.hist(sensor_data['value_c'], bins=50, alpha=0.7)
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Count')
    plt.title(label)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Load Specific Sensor Data

In [ ]:
# Get first sensor registration number
sensor_reg = df['sensor_registration'].iloc[0]

# Load all data for this sensor (across all deployments)
sensor_df = load_sensor_data(DB_PATH, sensor_reg)

print(f"Sensor: {sensor_reg}")
print(f"Deployments: {sensor_df['deployment_name'].unique()}")
print(f"Total readings: {len(sensor_df):,}")

sensor_df.head()

## 7. Time Range Queries

In [ ]:
# Query specific time range (example: first day of deployment)
start_time = df['time_utc'].min()
end_time = start_time + 86400  # +24 hours

day1_df = load_time_range_data(DB_PATH, start_time, end_time, DEPLOYMENT)

print(f"First day readings: {len(day1_df):,}")

# Plot first day
plt.figure(figsize=(14, 6))
day1_df['datetime_utc'] = pd.to_datetime(day1_df['time_utc'], unit='s', utc=True)

for sensor in day1_df['sensor_registration'].unique():
    sensor_data = day1_df[day1_df['sensor_registration'] == sensor]
    label = sensor_data['sensor_label'].iloc[0] if sensor_data['sensor_label'].iloc[0] else sensor
    plt.plot(sensor_data['datetime_utc'], sensor_data['value_c'], label=label, marker='.')

plt.xlabel('Time (UTC)')
plt.ylabel('Temperature (°C)')
plt.title(f'First 24 Hours - {DEPLOYMENT}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Convert to Local Time for Display

In [ ]:
from zoneinfo import ZoneInfo

# Convert UTC to Pittsburgh local time
df['datetime_local'] = pd.to_datetime(df['time_utc'], unit='s', utc=True).dt.tz_convert('America/New_York')

# Plot in local time
plt.figure(figsize=(14, 6))

for sensor in df['sensor_registration'].unique():
    sensor_data = df[df['sensor_registration'] == sensor]
    label = sensor_data['sensor_label'].iloc[0] if sensor_data['sensor_label'].iloc[0] else sensor
    plt.plot(sensor_data['datetime_local'], sensor_data['value_c'], label=label, alpha=0.7)

plt.xlabel('Time (Pittsburgh Local)')
plt.ylabel('Temperature (°C)')
plt.title(f'Temperature Data (Local Time) - {DEPLOYMENT}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Export Data

In [ ]:
# Export to CSV
output_file = f'{DEPLOYMENT}_temperatures.csv'
df.to_csv(output_file, index=False)
print(f"Data exported to: {output_file}")

## 10. Advanced: Resample to Hourly Averages

In [ ]:
# Resample to hourly averages for each sensor
hourly_data = []

for sensor in df['sensor_registration'].unique():
    sensor_data = df[df['sensor_registration'] == sensor].copy()
    sensor_data = sensor_data.set_index('datetime_utc')
    
    hourly = sensor_data['value_c'].resample('1H').mean()
    hourly_df = hourly.to_frame().reset_index()
    hourly_df['sensor'] = sensor
    hourly_data.append(hourly_df)

hourly_df = pd.concat(hourly_data, ignore_index=True)

# Plot hourly averages
plt.figure(figsize=(14, 6))

for sensor in hourly_df['sensor'].unique():
    sensor_data = hourly_df[hourly_df['sensor'] == sensor]
    plt.plot(sensor_data['datetime_utc'], sensor_data['value_c'], label=sensor, linewidth=2)

plt.xlabel('Time (UTC)')
plt.ylabel('Temperature (°C)')
plt.title(f'Hourly Average Temperatures - {DEPLOYMENT}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Original data points: {len(df):,}")
print(f"Hourly averages: {len(hourly_df):,}")